# Загрузка Pandas и очистка данных

In [1]:
import pandas as pd
import numpy as np
import re

from datetime import datetime

In [2]:
data = pd.read_csv('main_task.csv')

# Restaurant_id — идентификационный номер ресторана / сети ресторанов;
# City — город, в котором находится ресторан;
# Cuisine Style — кухня или кухни, к которым можно отнести блюда, предлагаемые в ресторане;
# Ranking — место, которое занимает данный ресторан среди всех ресторанов своего города;
# Rating — рейтинг ресторана по данным TripAdvisor (именно это значение должна будет предсказывать модель);
# Price Range — диапазон цен в ресторане;
# Number of Reviews — количество отзывов о ресторане;
# Reviews — данные о двух отзывах, которые отображаются на сайте ресторана;
# URL_TA — URL страницы ресторана на TripAdvisor;
# ID_TA — идентификатор ресторана в базе данных TripAdvisor.

data.head(2)

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032


## Смотрим информацию о датасете

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      40000 non-null  object 
 1   City               40000 non-null  object 
 2   Cuisine Style      30717 non-null  object 
 3   Ranking            40000 non-null  float64
 4   Rating             40000 non-null  float64
 5   Price Range        26114 non-null  object 
 6   Number of Reviews  37457 non-null  float64
 7   Reviews            40000 non-null  object 
 8   URL_TA             40000 non-null  object 
 9   ID_TA              40000 non-null  object 
dtypes: float64(3), object(7)
memory usage: 3.1+ MB


In [4]:
data.describe()

,Ranking,Rating,Number of Reviews
count,40000.000000,40000.000000,37457.000000
mean,3676.028525,3.993037,124.825480
std,3708.749567,0.668417,295.666352
min,1.000000,1.000000,2.000000
25%,973.000000,3.500000,9.000000
50%,2285.000000,4.000000,33.000000
75%,5260.000000,4.500000,115.000000
max,16444.000000,5.000000,9660.000000


## Убираем ненужные колонки

In [5]:
data = data.drop(['URL_TA', 'ID_TA'], axis=1)
data.columns

Index(['Restaurant_id', 'City', 'Cuisine Style', 'Ranking', 'Rating',
       'Price Range', 'Number of Reviews', 'Reviews'],
      dtype='object')

## Преобразуем цены

In [6]:
data['Price Range'].value_counts()

$$ - $$$    18412
$            6279
$$$$         1423
Name: Price Range, dtype: int64

In [7]:
def fix_price_range(dollars):
    if dollars == '$':
        return 'price lo'
    elif dollars == '$$$$':
        return 'price hi'
    else:
        return 'price mid'

In [8]:
data['Price Range'] = data['Price Range'].dropna().apply(fix_price_range)

In [9]:
data = data.join(pd.get_dummies(data['Price Range']))

In [10]:
data = data.drop('Price Range', axis=1)

## Обрабатываем города - ещё нужно придумать логику... может get_dummies?..

In [11]:
data['City'].value_counts()

London        5757
Paris         4897
Madrid        3108
Barcelona     2734
Berlin        2155
Milan         2133
Rome          2078
Prague        1443
Lisbon        1300
Vienna        1166
Amsterdam     1086
Brussels      1060
Hamburg        949
Munich         893
Lyon           892
Stockholm      820
Budapest       816
Warsaw         727
Dublin         673
Copenhagen     659
Athens         628
Edinburgh      596
Zurich         538
Oporto         513
Geneva         481
Krakow         443
Oslo           385
Helsinki       376
Bratislava     301
Luxembourg     210
Ljubljana      183
Name: City, dtype: int64

## Работаем с разнообразием кухонь

Здесь достаём их из списка, который находится в столбце "Cuisine Style"

In [12]:
def split_cuisines(cuisines:str) -> str:
    cui_list = list()
    for cui in re.findall("(?<=')[\w\s]+", cuisines):
        cui_list.append(cui)
    return '|'.join(cui_list)

Создаем отдельный столбец для преобразованных данных

In [13]:
data['Cuisine Style List'] = data['Cuisine Style'].dropna().apply(split_cuisines)

В следующий столбец запишем кол-во кухонь представленных в ресторане

In [14]:
data['Count Cuisines'] = data['Cuisine Style List'].dropna().apply(lambda x: len(str(x).split('|')))
data['Count Cuisines'] = data['Count Cuisines'].fillna(1)

In [15]:
data.loc[:, ['Cuisine Style List', 'Count Cuisines']]['Count Cuisines'].mean()

2.6224

Применим get_dummies для создания дополнительных 125 столбцов по кол-ву возможных кухонь.

In [16]:
data = data.join(data['Cuisine Style List'].str.get_dummies(sep='|'))

In [17]:
cols = data.columns[:11]
cols

Index(['Restaurant_id', 'City', 'Cuisine Style', 'Ranking', 'Rating',
       'Number of Reviews', 'Reviews', 'price hi', 'price lo', 'price mid',
       'Cuisine Style List'],
      dtype='object')

In [18]:
cuisines_cols = data.columns[11:]
cuisines_cols

Index(['Count Cuisines', 'Afghani', 'African', 'Albanian', 'American',
       'Arabic', 'Argentinean', 'Armenian', 'Asian', 'Australian',
       ...
       'Ukrainian', 'Uzbek', 'Vegan Options', 'Vegetarian Friendly',
       'Venezuelan', 'Vietnamese', 'Welsh', 'Wine Bar', 'Xinjiang', 'Yunnan'],
      dtype='object', length=126)

In [19]:
count_cuisines = dict()
for col in cuisines_cols:
    count_cuisines[col] = data.loc[data[col] == 1, [col]].count()[0]

total_cuisines = pd.Series(count_cuisines)

In [20]:
total_cuisines.sort_values(ascending=False).head()

Count Cuisines         16547
Vegetarian Friendly    11189
European               10060
Mediterranean           6277
Italian                 5964
dtype: int64

## Работаем с датами отзывов

Сначала получим раннюю из двух дат

In [21]:
def convert_dt(line):
    dd = re.compile('\d{2}/\d{2}/\d{4}')
    dates = dd.findall(line)
    if not dates:
        return None
    dlist = list()
    for date in dates:
        dlist.append(datetime.strptime(date, '%m/%d/%Y'))
    if len(dlist) > 1:
        return min(dlist)
    else:
        return None

In [22]:
data['prev review'] = data['Reviews'].dropna().apply(convert_dt)

А теперь дату последнего обзора

In [23]:
def convert_dt(line):
    dd = re.compile('\d{2}/\d{2}/\d{4}')
    dates = dd.findall(line)
    if not dates:
        return None
    dlist = list()
    for date in dates:
        dlist.append(datetime.strptime(date, '%m/%d/%Y'))
    return max(dlist)

In [24]:
data['last review'] = data['Reviews'].dropna().apply(convert_dt)

Посчитаем разницу в днях между двумя датами

In [25]:
def count_days(line):
    dd = re.compile('\d{2}/\d{2}/\d{4}')
    dates = dd.findall(line)
    if not dates:
        return None
    dlist = list()
    for date in dates:
        dlist.append(datetime.strptime(date, '%m/%d/%Y'))
    if len(dlist) == 2:
        diff = max(dlist) - min(dlist)
        return diff.days
    else:
        return None

In [26]:
data['diff days'] = data['Reviews'].dropna().apply(count_days)

In [27]:
data[['diff days']].describe()

,diff days
count,28973.000000
mean,141.712180
std,221.165893
min,0.000000
25%,20.000000
50%,67.000000
75%,173.000000
max,3207.000000


Посмотрим топ 10 самых больших значений разницы между обзорами.

In [28]:
data.loc[data['diff days'] > 0, ['prev review', 'last review', 'diff days']].sort_values(by='diff days').tail(10)

,prev review,last review,diff days
34381,2008-01-23,2016-02-10,2940.0
12559,2008-06-06,2016-06-27,2943.0
11140,2007-11-14,2015-12-28,2966.0
6958,2008-05-02,2016-08-23,3035.0
2028,2008-07-01,2016-10-27,3040.0
19438,2008-10-25,2017-04-15,3094.0
8356,2008-01-06,2016-06-28,3096.0
39997,2008-04-12,2016-11-03,3127.0
10997,2008-03-25,2016-10-26,3137.0
7990,2007-12-22,2016-10-02,3207.0


Уберем ненужные столбцы, из которых мы уже вытянули информацию.

In [29]:
data = data.drop(['Cuisine Style', 'Cuisine Style List'], axis=1)

Поменяем некоторые столбцы местами для удобства

In [30]:
new_col_order = list()
for i in range(9):
    new_col_order.append(data.columns[i])
for j in range(1, 4):
    new_col_order.append(data.columns[-j])
for k in range(9, len(data.columns)-3):
    new_col_order.append(data.columns[k])

In [31]:
data = data[new_col_order]

In [32]:
data[data.columns[:12]].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Restaurant_id      40000 non-null  object        
 1   City               40000 non-null  object        
 2   Ranking            40000 non-null  float64       
 3   Rating             40000 non-null  float64       
 4   Number of Reviews  37457 non-null  float64       
 5   Reviews            40000 non-null  object        
 6   price hi           40000 non-null  uint8         
 7   price lo           40000 non-null  uint8         
 8   price mid          40000 non-null  uint8         
 9   diff days          28973 non-null  float64       
 10  last review        33529 non-null  datetime64[ns]
 11  prev review        28973 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(4), object(3), uint8(3)
memory usage: 2.9+ MB


## Подготовим данные к обучению

In [33]:
df = data[data.columns[:12]].drop(['Restaurant_id', 'City', 'Reviews', 'prev review', 'last review'], axis=1)

In [34]:
df.columns

Index(['Ranking', 'Rating', 'Number of Reviews', 'price hi', 'price lo',
       'price mid', 'diff days'],
      dtype='object')

In [35]:
df = df.dropna()
df.shape

(28973, 7)

# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [36]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df.drop('Rating', axis = 1)
y = df['Rating']

In [37]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [38]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [39]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [40]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [41]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.34681874654886796
